In [3]:
import logging, os, sys
import rpy2
from rpy2 import robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
import anndata2ri
import anndata
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
#import rpy2.rinterface_lib.callbacks
import anndata2ri
from matplotlib import rcParams
from matplotlib import colors

In [4]:
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
sc.set_figure_params(dpi=200, dpi_save=300)
sc.settings.figdir = 'markers'
sc.settings.autoshow = False

In [5]:
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
#rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)
# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()

The `sinfo` package has changed name and is now called `session_info` to become more discoverable and self-explanatory. The `sinfo` PyPI package will be kept around to avoid breaking old installs and you can downgrade to 0.3.2 if you want to use it without seeing this message. For the latest features and bug fixes, please install `session_info` instead. The usage and defaults also changed slightly, so please review the latest README at https://gitlab.com/joelostblom/session_info.
-----
anndata     0.7.6
scanpy      1.8.0
sinfo       0.3.4
-----
PIL                 8.2.0
anndata2ri          1.0.6
anyio               NA
attr                21.2.0
babel               2.9.1
backcall            0.2.0
brotli              NA
certifi             2021.05.30
cffi                1.14.5
chardet             4.0.0
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.1
decorator           5.0.9
dunamai             1.5.5
get_version         3.2
h5py                3.2.1
idna      

In [6]:
adata = sc.read('filtered.h5ad')
print(adata)
print('Number of cells: {:d}'.format(adata.n_obs))

AnnData object with n_obs × n_vars = 19950 × 18478
    obs: 'replicate', 'type', 'n_genes', 'n_counts', 'batch'
    var: 'n_cells'
    uns: 'replicate_colors', 'type_colors'
Number of cells: 19950


In [7]:
adata_pp = adata.copy()
sc.pp.normalize_per_cell(adata_pp, counts_per_cell_after=1e6)
sc.pp.log1p(adata_pp)
sc.pp.pca(adata_pp, n_comps=20, svd_solver='arpack')
sc.pp.neighbors(adata_pp)
sc.tl.leiden(adata_pp, key_added='groups', resolution=0.3)

normalizing by total count per cell
    finished (0:00:01): normalized adata.X and added    'n_counts', counts per cell before normalization (adata.obs)
computing PCA
    with n_comps=20
    finished (0:00:09)
computing neighbors
    using 'X_pca' with n_pcs = 20
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:27)
running Leiden clustering
    finished: found 8 clusters and added
    'groups', the cluster labels (adata.obs, categorical) (0:00:02)


In [8]:
input_groups = adata_pp.obs['groups']
data_mat = adata.X.T
data_mat = sp.sparse.csc_matrix(data_mat)

In [9]:
# -i data_mat -i input_groups -o size_factors
scran = importr('scran')
size_factors = scran.computeSumFactors(data_mat, clusters=input_groups, **{'min.mean': 0.1})
print(size_factors)

PackageNotInstalledError: The R package "scran" is not installed.